# From .txt(raw file) to .csv file(x,y,theta)

### "Course" field in dynamic data is used to replace HDT 

### Dependencies

In [1]:
import os
import csv
import numpy as np

c:\Users\happy\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\happy\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\happy\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# dir_path = r"Dynamic Logs for Testing"
# input_dir_path = os.path.join(dir_path, "Special_CanBeBoth")
# output_dir_path = os.path.join(dir_path, "Special_Processed")

dir_path = r"C:\Users\happy\OneDrive\Documents\Studies\NSTC project\Log\Bad"
input_dir_path = os.path.join(dir_path)
output_dir_path = os.path.join(dir_path)

if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

vecX = np.array([7.260e-06, 6.136e-06])
vecY = np.array([-6.840e-06, 6.544e-06])
basis = np.column_stack((vecX, vecY))
offset = np.array([-121.5409683, -25.0129261])

In [3]:
texts_array = []
textFiles = [f for f in os.listdir(input_dir_path) if os.path.isfile(os.path.join(input_dir_path, f)) and f.endswith(".txt")]
print(textFiles)

['Firebird_P_NMEA_Log_USB_COM_230324_171812.txt', 'Firebird_P_NMEA_Log_USB_COM_230324_172849.txt']


In [4]:
texts_array.clear()
for tFile in textFiles:
    fullFile = os.path.join(input_dir_path, tFile)
    with open(fullFile, 'r') as file:
        file_content = file.readlines()
        texts_array.append(file_content)

# texts_array[0]
print(len(texts_array))

2


In [5]:
i = 0
for texts in texts_array:
    # Remove the last line
    texts.pop()
    file_content = texts

    # Extract lines starting with $GNGGA or $GPHDT
    extracted_lines = [line for line in file_content if line.startswith('$GNGGA') or line.startswith('$GNRMC')]
    
    # Ensure the first line starts with $GNGGA and the last line with $GPHDT
    if not extracted_lines or not extracted_lines[0].startswith('$GNGGA'):
        extracted_lines.pop(0);
    if not extracted_lines or not extracted_lines[-1].startswith('$GNRMC'):
        extracted_lines.pop();
    
    # Ensure there are an even number of extracted lines
    if len(extracted_lines) % 2 != 0:
        print(f"Odd number of extracted lines in data_array[{i}][].")
    # else:
    #     print("Success.")
    
    # Replace the element in data_array with the extracted lines
    texts_array[i] = extracted_lines
    i += 1

In [6]:
# Convert dd.mmmmmmmmm... to dd.dddddddd
def ddm2ddd(n):
    d = int(n)
    m_in_d = (n - d) * 5 / 3 # *(100/60)
    return round(d + m_in_d, 8)

In [7]:
def angleConversion(h):
    # 0 <-> 45, 45 <-> 0, 90 <-> 315, 135 <-> 270, 180 <-> 225 ... 
    if h == -1:
        return h
    
    return round((405 - h) % 360, 3)

# def generate_flat_random_numbers(num_samples, range_width=2.6):
#     flat_values = np.random.uniform(low=-range_width/2, high=range_width/2, size=num_samples)
#     flat_values -= np.mean(flat_values) # Adjust the values to have an average of zero
#     rounded_values = np.round(flat_values, 3)
#     return rounded_values

# range_width = 2.6 # +-k -> 2k (+-1.3)

def LL2XY(Lat, Lon):
    v = np.array([Lon + offset[0], Lat + offset[1]])
    X, Y = np.linalg.solve(basis, v)
    return round(X, 5), round(Y, 5)

# def convHDT(HDT, truth):
#     if HDT != -1:
#         return angleConversion(HDT)
#     else: # Need to generate a faux HDT
#         r = np.random.uniform(-5, 5)
#         t = truth + r # No need angConv since it's already self-defined
#         if t < 0:
#             t += 360
#         return round(t, 3)
        

In [9]:
processed_data = []

idx = 0
for file_content in texts_array:
    i = 0
    n = len(file_content) 
    processed_data.clear()
    # Process pairs of lines starting with "$GNGGA" and "$GPHDT"
    for k in range(0, n, 2):
        if k + 1 < n and file_content[k].startswith('$GNGGA') and file_content[k + 1].startswith('$GNRMC'):
            gngga_line = file_content[k].split(',')
            gnrmc_line = file_content[k + 1].split(',')
            
            # Extract the desired fields from $GNGGA and $GPHDT lines
            if len(gngga_line) >= 5 and len(gnrmc_line) >= 3:
                Lat = ddm2ddd(float(gngga_line[2]) / 100)
                Lon = ddm2ddd(float(gngga_line[4]) / 100)
                x, y = LL2XY(Lat, Lon)
                ind = gngga_line[6]

                # Now we use RMC course as HDT
                try:
                    HDT = float(gnrmc_line[8]) if gnrmc_line[8] else -1
                except ValueError:
                    HDT = -1  # Assign a default value if conversion fails
                t = angleConversion(HDT)
                processed_data.append([i, Lat, Lon, HDT, ind, x, y, t])
        i += 1
    
    filename = textFiles[idx] + '.csv'
    output_csv_file = os.path.join(output_dir_path, filename)
    # print(idx)
    # print(filename)
    with open(output_csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(processed_data)

    idx += 1